<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Add note to contact
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Add+note+to+contact:+Error+short+description">Bug report</a>

**Tags:** #hubspot #sales #crm #engagements #notes #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-08-29 (Created: 2023-08-29)

**Description:** This notebook demonstrates how to add a note to one a several contacts in HubSpot using their contact ID. This can be particularly useful for recording details about a contact's engagement with your published content or their usage of your product.

**References:**
- [HubSpot API - Notes](https://developers.hubspot.com/docs/api/crm/notes)

## Input

### Import libraries

In [ ]:
import requests
import naas
from datetime import datetime, date, timezone

### Setup variables
[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `body`: This variable stores the message that will be added to the note.
- `contact_ids`: This variable stores the list of HubSpot contact ID that will be associated to the note.
- `note_datetime`: This variable stores the datetime object of the note. If None 'note' date will be now.

In [ ]:
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
body = "My new note!"
contact_id = ['307851']
note_timestamp = datetime.strptime('2023-08-24 03:54:57', "%Y-%m-%d %H:%M:%S") or datetime(2023, 8, 17, 16, 5, 5)

## Model

### Create note associated to contact

In [ ]:
def create_note(
    api_key,
    body,
    object_datetime=None,
    contact_ids=[],
):
    # Init
    data = []
    
    # Get the current timestamp in UTC
    if not object_datetime:
        object_datetime = datetime.utcnow()
        
    hs_timestamp = object_datetime.replace(tzinfo=timezone.utc).strftime("%s") + "000"
    
    # Create contact asso
    contacts = []
    for contact_id in contact_ids:
        contacts.append(
            {
                "to": {"id": contact_id},
                "types": [
                    {
                        "associationCategory": "HUBSPOT_DEFINED",
                        "associationTypeId": 202
                    }
                ]
            }
        )
                
    # Requests
    payload = {
        "properties":
        {
            "hs_note_body": body,
            "hs_timestamp": hs_timestamp
        },
        "associations": contacts
    }
    headers = {
        'accept': "application/json",
        'content-type': "application/json",
        'authorization': f"Bearer {api_key}"
    }
    url = "https://api.hubapi.com/crm/v3/objects/notes"
    
    # Response
    res = requests.post(url, headers=headers, json=payload)
    if res.status_code == 201:
        data = res.json()
        print("✅ Note successfully created:", data.get('id'))
    else:
        print(res)
        print(res.text)
    return data


note = create_note(
    hs_access_token,
    body,
    note_timestamp,
    contact_id
)
note

## Output

### Display result

In [ ]:
note